In [72]:
import csv
from bs4 import BeautifulSoup


In [73]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [74]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


Startup the webdriver


In [75]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\SAANIY~1\AppData\Local\Temp/ipykernel_36520/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [104]:
url = 'https://www.amazon.com'
driver.get(url)

WebDriverException: Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x01121ED3+2236115]
	Ordinal0 [0x010B92F1+1807089]
	Ordinal0 [0x00FC66FD+812797]
	Ordinal0 [0x00FB798A+752010]
	Ordinal0 [0x00FB77D8+751576]
	Ordinal0 [0x00FC7B63+818019]
	Ordinal0 [0x0102518A+1200522]
	Ordinal0 [0x010118A6+1120422]
	Ordinal0 [0x00FEA73D+960317]
	Ordinal0 [0x00FEB71F+964383]
	GetHandleVerifier [0x013CE7E2+2743074]
	GetHandleVerifier [0x013C08D4+2685972]
	GetHandleVerifier [0x011B2BAA+532202]
	GetHandleVerifier [0x011B1990+527568]
	Ordinal0 [0x010C080C+1837068]
	Ordinal0 [0x010C4CD8+1854680]
	Ordinal0 [0x010C4DC5+1854917]
	Ordinal0 [0x010CED64+1895780]
	BaseThreadInitThunk [0x75D07BA9+25]
	RtlInitializeExceptionChain [0x76FBBB3B+107]
	RtlClearBits [0x76FBBABF+191]


In [ ]:
def get_url(search_term):
    template='https://www.amazon.com/s?k={}&crid=XPS7LD4VUAAH&sprefix=bags%2Caps%2C263&ref=nb_sb_noss_1'
    search_term=search_term.replace('','+')
    return template.format(search_term)

In [ ]:
url=get_url('bags')
print(url)

In [ ]:
driver.get(url)

 Extract the collection

In [ ]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
results = soup.find_all('div',{'data-component-type': 's-search-result'})

In [ ]:
len(results)

Prototype the record

In [94]:
item = results[0]

In [95]:
atag = item.h2.a

In [96]:
description = atag.text.strip()

In [97]:
url = 'https://www.amazon.com' + atag.get('href')

In [98]:
price_parent = item.find('span','a-price')

In [99]:
price = price_parent.find('span','a-offscreen').text

AttributeError: 'NoneType' object has no attribute 'find'

In [100]:
rating = item.i.text

In [101]:
review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text

AttributeError: 'NoneType' object has no attribute 'text'

Generalize the pattern

In [102]:
def extract_record(item):
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    #price
    price_parent = item.find('span','a-price')
    price = price_parent.find('span','a-offscreen').text
    
    
    #rank and rating
    rating = item.i.text
    review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    
    result = (description,price,rating,review_count,url)
    
    return result

In [103]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})
for item in results:
    records.append(extract_record(item))

AttributeError: 'NoneType' object has no attribute 'find'

Error Handling

In [ ]:
def extract_record(item):
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    try:
        #rank and rating
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    result = (description,price,rating,review_count,url)
    
    return result

In [ ]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})


for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [ ]:
records[0]

In [ ]:
for row in records:
    print(row[1])

Getting the next page

In [ ]:
 def get_url(search_term):
    template='https://www.amazon.com/s?k={}&crid=XPS7LD4VUAAH&sprefix=bags%2Caps%2C263&ref=nb_sb_noss_1'
    search_term=search_term.replace('','+')
    
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    
    url += '&page={}'
    
    return url

Putting it all together

In [91]:
import csv
from bs4 import BeautifulSoup


def get_url(search_term):
    template='https://www.amazon.com/s?k={}&crid=XPS7LD4VUAAH&sprefix=bags%2Caps%2C263&ref=nb_sb_noss_1'
    search_term=search_term.replace('','+')
    
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    
    url += '&page{}'
    
    return url


def extract_record(item):
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    try:
        #rank and rating
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    result = (description,price,rating,review_count,url)
    
    return result



def main(search_term):
    
    
    #startup the webdriver
    from selenium import webdriver
    from webdriver_manager.chrome import ChromeDriverManager
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    
    record = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source,'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    #save data to csv file
    
    with open('results1.csv','w',newline ='',encoding ='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','ReviewCount','Url'])
        writer.writerows(records)

In [92]:
main('bags')

C:\Users\SAANIY~1\AppData\Local\Temp/ipykernel_36520/2191235769.py:51: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
